In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
openai = OpenAI()

In [4]:
caminho_csv = "Data\\DadosProjetos.csv"
df = pd.read_csv(caminho_csv)

In [5]:
def gerar_prompt(dado_dict):
    base_prompt = f"""Given the structured project data dictionary called `dado`, analyze and recommend one non-functional requirement (NFR) that fits the scenario. Only recommend NFRs belonging to the following types and attributes:

- Performance: response time, capacity, transit delay, efficiency compliance  
- Reliability: availability, integrity, fault tolerance, recoverability  
- Security: confidentiality, access control, authentication  

Follow this step-by-step reasoning:

1. Read the project data from the `dado` dictionary:
   - Functional requirement (fr): {dado_dict["fr"]}
   - Domain: {dado_dict["dominio"]}
   - Platform: {dado_dict["plataforma"]}
   - Architecture: {dado_dict["arquitetura"]}
   - Module: {dado_dict["modulo"]}
   - Operation: {dado_dict["operacao"]}
   - Task Description: {dado_dict["tarefa_mapeada"]}
   - Original task details: {dado_dict["tarefa_original"]}
   - Technologies: {dado_dict["tecnologias"]}

2. Understand the functional requirement (FR) and its sensitivity or business implications.

3. Analyze each field of the `dado` dictionary to understand its significance for NFR selection:

   - `fr`: Describes the functional requirement or user story. It is essential for understanding what the system should accomplish.
   - `dominio`: Represents the business domain (e.g., Finance, Health), which influences NFR criticality (e.g., confidentiality in Finance).
   - `plataforma`: Indicates the type of system (Web, Mobile, etc.). This affects NFRs like latency (Performance) and fault recovery (Reliability).
   - `arquitetura`: Specifies the software architecture (e.g., client-server, microservices). This informs needs like fault tolerance or throughput.
   - `modulo`: Identifies the specific part of the system (e.g., Authentication, Reporting) involved, which may have different security or integrity requirements.
   - `operacao`: Describes the functional action (e.g., retrieve, validate). Certain operations imply stricter performance or access control.
   - `tarefa_mapeada`: A concise description of the concrete task to implement the feature (e.g., "Criar serviço Auth (Client)", "Validar e-mail e senha informado"). This gives insight into technical effort and risk.
   - `tarefa_original`: A more detailed textual description, possibly including rules or business constraints.
   - `tecnologias`: Lists technologies used (e.g., Angular, Springboot), which may bring built-in NFR implications (e.g., security libraries, performance overhead).

4. Based on the full context, select the most appropriate NFR type and attribute from the allowed list.

5. Draft one clear and relevant NFR sentence aligned with the chosen attribute.  
   - Format: "The system must ensure the [ATTRIBUTE] by [MEASURE OR CONDITION]."

6. Format the final output as JSON using the actual task ID from the dataset:

[
  {{
    "Tarefa_ID": "{dado_dict['tarefa_mapeada']}",
    "NFR_Tipo": "[Performance|Reliability|Security]",
    "NFR_Atributo": "[attribute from allowed list]",
    "NFR_Sentença": "..."
  }}
]

Now generate the recommendation and return only a valid JSON array, without comments, explanations, or any extra text.
"""
    return base_prompt

In [6]:
resultados = []

try:
    for _, row in df.iterrows():
        tecnologias = []

        for col in ["Linguagem", "Framework", "API", "Outras_Tags"]:
            valor = row[col]
            if pd.notna(valor) and valor.strip():
                tecnologias.extend(valor.split(","))

        tecnologias = [tech.strip() for tech in tecnologias if tech.strip()]

        dado = {
            "fr": row["Descritivo_da_US"],
            "dominio": row["Dominio"],
            "plataforma": row["Plataforma"],
            "arquitetura": row["Arquitetura"],
            "modulo": row["Modulo"],
            "operacao": row["Operacao"],
            "tarefa_mapeada": row["Tarefa_mapeada"],
            "tarefa_original": row["Tarefa_original"],
            "tecnologias": tecnologias
        }

        prompt = gerar_prompt(dado)

        response = openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": "You are a requirements engineer. Use chain-of-thought reasoning to analyze the user's input."},
                {"role": "user", "content": prompt}
            ]
        )

        print("\n--- RESPOSTA ---\n")
        resposta_str = response.choices[0].message.content.strip()
        print(resposta_str)

        # Tenta converter a resposta para objeto JSON
        try:
            resposta_json = json.loads(resposta_str)
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar a resposta como JSON: {e}")
            resposta_json = [{"erro": "Resposta inválida", "resposta_bruta": resposta_str}]

        resultados.append({
            "entrada": dado,
            "resposta": resposta_json
        })

        with open("openai_results.json", "w", encoding="utf-8") as f:
            json.dump(resultados, f, indent=2, ensure_ascii=False)

        print("NFR Finalizado")
        print("----------------------------------")

except Exception as e:
    print(f"Erro ocorrido: {e}")
    print("Salvando resultados parciais...")

finally:
    print(f"Resultados salvos com {len(resultados)} entradas.")


--- RESPOSTA ---

[
  {
    "Tarefa_ID": "Adicionar atributo de proteção no provider",
    "NFR_Tipo": "Security",
    "NFR_Atributo": "access control",
    "NFR_Sentença": "The system must ensure access control by enforcing user permission validation on every sensitive operation in the Authentication module."
  }
]
NFR Finalizado
----------------------------------

--- RESPOSTA ---

[
  {
    "Tarefa_ID": "Validar permissão do usuário na aplicação",
    "NFR_Tipo": "Security",
    "NFR_Atributo": "access control",
    "NFR_Sentença": "The system must ensure access control by enforcing that all user permissions for web applications start with ROLE_, complying with Spring Security standards."
  }
]
NFR Finalizado
----------------------------------

--- RESPOSTA ---

[
  {
    "Tarefa_ID": "Criar tela listagem",
    "NFR_Tipo": "Security",
    "NFR_Atributo": "access control",
    "NFR_Sentença": "The system must ensure the access control by restricting the user permissions list to auth